In [ ]:
import os
import json
from pathlib import Path
from tqdm import tqdm

def create_medical_manifest(base_path, output_file="medical_manifest.jsonl"):
    """
    비대면 진료 음성 데이터에서 manifest 파일을 생성합니다.
    
    데이터 구조:
    - medv/: 텍스트 정보 (JSON/TXT) - 역할별 폴더 (간호사/의사/환자)
    - nur/: 간호사 WAV 파일들
    - doc/: 의사 WAV 파일들  
    - pat/: 환자 WAV 파일들
    
    Args:
        base_path (str): 기본 경로 (Validation 폴더 경로)
        output_file (str): 출력할 manifest 파일명
    """
    
    base_dir = Path(base_path)
    medv_dir = base_dir / "medv"
    
    # 역할별 오디오 디렉토리 매핑
    audio_dirs = {
        "간호사": base_dir / "nur",
        "의사": base_dir / "doc", 
        "환자": base_dir / "pat"
    }
    
    manifest_data = []
    
    print("📂 파일 수 계산 중...")
    
    # 모든 JSON 파일 수집
    all_json_files = []
    for role_dir in medv_dir.iterdir():
        if role_dir.is_dir():
            for speaker_dir in role_dir.iterdir():
                if speaker_dir.is_dir():
                    json_files = list(speaker_dir.glob("*.json"))
                    all_json_files.extend(json_files)
    
    print(f"총 {len(all_json_files):,}개의 JSON 파일을 처리합니다.")
    
    missing_audio_files = []
    
    # 프로그레스 바를 사용하여 파일 처리
    with tqdm(total=len(all_json_files), desc="Manifest 생성") as pbar:
        for json_file in all_json_files:
            # 역할과 화자 정보 추출
            role_name = json_file.parent.parent.name
            speaker_name = json_file.parent.name
            
            # JSON 파일 읽기
            with open(json_file, 'r', encoding='utf-8') as f:
                label_data = json.load(f)
            
            # 해당하는 WAV 파일 경로 찾기
            audio_filename = json_file.stem + '.wav'
            audio_dir = audio_dirs[role_name]
            audio_path = audio_dir / speaker_name / audio_filename
            
            # 오디오 파일 존재 확인
            if not audio_path.exists():
                missing_audio_files.append(str(audio_path))
                pbar.update(1)
                continue
            
            # 전사 정보 추출
            transcript = label_data.get('전사정보', {}).get('LabelText', '')
            
            # 파일 정보 추출
            file_info = label_data.get('파일정보', {})
            duration_str = file_info.get('FileLength', '0')
            duration = float(duration_str) if duration_str else 0.0
            
            # 화자 정보 추출
            speaker_info = label_data.get('화자정보', {})
            gender = speaker_info.get('Gender', '')
            age = speaker_info.get('Age', '')
            region = speaker_info.get('Region', '')
            dialect = speaker_info.get('Dialect', '')
            
            # Manifest 엔트리 생성
            manifest_entry = {
                "audio_filepath": str(audio_path),
                "text": transcript,
                "duration": duration,
                "speaker_id": speaker_name,
                "role": role_name,
                "gender": gender,
                "age": age,
                "region": region,
                "dialect": dialect
            }
            
            manifest_data.append(manifest_entry)
            pbar.update(1)
    
    # Manifest 파일 저장
    print(f"\n💾 {output_file}에 저장 중...")
    with open(output_file, 'w', encoding='utf-8') as f:
        for entry in manifest_data:
            f.write(json.dumps(entry, ensure_ascii=False) + '\n')
    
    print(f"✅ Manifest 파일이 생성되었습니다: {output_file}")
    print(f"📊 총 {len(manifest_data):,}개의 음성 파일이 처리되었습니다.")
    
    if missing_audio_files:
        print(f"⚠️  {len(missing_audio_files):,}개의 오디오 파일을 찾을 수 없습니다.")
        print("   (텍스트는 있지만 해당 오디오 파일이 없는 경우)")
    
    # 통계 정보 출력
    roles = {}
    for entry in manifest_data:
        role = entry['role']
        if role not in roles:
            roles[role] = 0
        roles[role] += 1
    
    print("\n📈 역할별 통계:")
    for role, count in roles.items():
        print(f"  {role}: {count:,}개")
    
    return manifest_data

# 실행
base_path = "AI_Hub_비대면진료/비대면_진료를_위한_의료진_및_환자_음성/Validation"
manifest_data = create_medical_manifest(base_path)


📂 파일 수 계산 중...
총 141,819개의 JSON 파일을 처리합니다.


Manifest 생성:  27%|██▋       | 37668/141819 [01:34<04:20, 399.07it/s]


KeyboardInterrupt: 

In [ ]:
# 생성된 데이터 샘플 확인
if 'manifest_data' in locals() and manifest_data:
    print("📝 생성된 데이터 샘플 (처음 5개):")
    print("=" * 80)
    
    for i, entry in enumerate(manifest_data[:5]):
        print(f"\n[{i+1}] {entry['role']} - {entry['speaker_id']}")
        print(f"   텍스트: {entry['text']}")
        print(f"   음성파일: {entry['audio_filepath']}")
        print(f"   길이: {entry['duration']}초")
        print(f"   화자정보: {entry['gender']}, {entry['age']}, {entry['region']}, {entry['dialect']}")
        
    print(f"\n📊 전체 데이터 개수: {len(manifest_data):,}개")
else:
    print("먼저 첫 번째 셀을 실행하여 manifest 데이터를 생성해주세요.")


In [ ]:
# 상세 분석 및 필터링 기능
import pandas as pd

def analyze_manifest_data(manifest_data):
    """생성된 manifest 데이터를 분석합니다."""
    
    df = pd.DataFrame(manifest_data)
    
    print("📊 상세 통계 분석")
    print("=" * 80)
    
    # 역할별 통계
    print("\n🎭 역할별 분포:")
    role_counts = df['role'].value_counts()
    for role, count in role_counts.items():
        print(f"  {role}: {count:,}개 ({count/len(df)*100:.1f}%)")
    
    # 성별 분포
    print("\n👥 성별 분포:")
    gender_counts = df['gender'].value_counts()
    for gender, count in gender_counts.items():
        print(f"  {gender}: {count:,}개 ({count/len(df)*100:.1f}%)")
    
    # 연령대 분포
    print("\n🎂 연령대 분포:")
    age_counts = df['age'].value_counts()
    for age, count in age_counts.items():
        print(f"  {age}: {count:,}개 ({count/len(df)*100:.1f}%)")
    
    # 지역별 분포
    print("\n🗺️ 지역별 분포:")
    region_counts = df['region'].value_counts()
    for region, count in region_counts.items():
        print(f"  {region}: {count:,}개 ({count/len(df)*100:.1f}%)")
    
    # 방언별 분포
    print("\n🗣️ 방언별 분포:")
    dialect_counts = df['dialect'].value_counts()
    for dialect, count in dialect_counts.items():
        print(f"  {dialect}: {count:,}개 ({count/len(df)*100:.1f}%)")
    
    # 음성 길이 통계
    print("\n⏱️ 음성 길이 통계:")
    print(f"  평균: {df['duration'].mean():.2f}초")
    print(f"  최소: {df['duration'].min():.2f}초")
    print(f"  최대: {df['duration'].max():.2f}초")
    print(f"  전체: {df['duration'].sum():.2f}초 ({df['duration'].sum()/3600:.2f}시간)")
    
    # 텍스트 길이 통계
    df['text_length'] = df['text'].str.len()
    print("\n📝 텍스트 길이 통계:")
    print(f"  평균: {df['text_length'].mean():.1f}자")
    print(f"  최소: {df['text_length'].min()}자")
    print(f"  최대: {df['text_length'].max()}자")
    
    # 화자별 발화 수 통계
    speaker_counts = df['speaker_id'].value_counts()
    print("\n🎤 화자별 발화 수 통계:")
    print(f"  평균: {speaker_counts.mean():.1f}개")
    print(f"  최소: {speaker_counts.min()}개")
    print(f"  최대: {speaker_counts.max()}개")
    print(f"  총 화자 수: {len(speaker_counts)}명")
    
    return df

def create_filtered_manifest(df, filters, output_file):
    """조건에 맞는 데이터만 필터링하여 새로운 manifest 파일 생성"""
    
    filtered_df = df.copy()
    
    # 필터 적용
    for key, values in filters.items():
        if key in df.columns:
            if isinstance(values, list):
                filtered_df = filtered_df[filtered_df[key].isin(values)]
            else:
                filtered_df = filtered_df[filtered_df[key] == values]
    
    print(f"필터링 결과: {len(filtered_df):,}개 데이터")
    
    # 필터링된 데이터를 manifest 파일로 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        for _, row in filtered_df.iterrows():
            entry = row.to_dict()
            # text_length 컬럼 제거 (manifest에 불필요)
            if 'text_length' in entry:
                del entry['text_length']
            f.write(json.dumps(entry, ensure_ascii=False) + '\n')
    
    print(f"필터링된 manifest 파일 저장: {output_file}")
    
    return filtered_df

# 분석 실행
if 'manifest_data' in locals() and manifest_data:
    df = analyze_manifest_data(manifest_data)
else:
    print("먼저 manifest 데이터를 생성해주세요.")


In [ ]:
# 필터링 예시
if 'df' in locals():
    print("🔍 필터링 예시")
    print("=" * 80)
    
    # 예시 1: 여성 의사 데이터만 추출
    filters_female_doctor = {
        'role': '의사',
        'gender': 'Female'
    }
    print("\n1️⃣ 여성 의사 데이터 추출")
    if len(df[(df['role'] == '의사') & (df['gender'] == 'Female')]) > 0:
        filtered_df1 = create_filtered_manifest(df, filters_female_doctor, "female_doctor_manifest.jsonl")
    else:
        print("여성 의사 데이터가 없습니다.")
    
    # 예시 2: 20-30대 환자 데이터만 추출
    filters_young_patient = {
        'role': '환자',
        'age': ['20~29', '30~39']
    }
    print("\n2️⃣ 20-30대 환자 데이터 추출")
    young_patients = df[(df['role'] == '환자') & (df['age'].isin(['20~29', '30~39']))]
    if len(young_patients) > 0:
        filtered_df2 = create_filtered_manifest(df, filters_young_patient, "young_patient_manifest.jsonl")
    else:
        print("20-30대 환자 데이터가 없습니다.")
    
    # 예시 3: 특정 지역 데이터만 추출
    available_regions = df['region'].unique()
    print(f"\n3️⃣ 이용 가능한 지역: {list(available_regions)}")
    
    if len(available_regions) > 0:
        target_region = available_regions[0]  # 첫 번째 지역 선택
        filters_region = {
            'region': target_region
        }
        print(f"'{target_region}' 지역 데이터 추출")
        filtered_df3 = create_filtered_manifest(df, filters_region, f"{target_region.replace('/', '_')}_region_manifest.jsonl")
    
    # 예시 4: 간호사 + 전라 방언
    filters_nurse_dialect = {
        'role': '간호사',
        'dialect': '전라'
    }
    print("\n4️⃣ 간호사 + 전라 방언 데이터 추출")
    nurse_dialect = df[(df['role'] == '간호사') & (df['dialect'] == '전라')]
    if len(nurse_dialect) > 0:
        filtered_df4 = create_filtered_manifest(df, filters_nurse_dialect, "nurse_jeolla_manifest.jsonl")
    else:
        print("간호사 + 전라 방언 데이터가 없습니다.")
        
else:
    print("먼저 분석을 실행해주세요.")


In [1]:
# 개선된 버전 - 진행률 표시 및 오류 처리 포함
import os
import json
from pathlib import Path
from tqdm import tqdm

def create_manifest_with_progress(base_path, output_file="manifest.jsonl"):
    """
    진행률 표시와 함께 manifest 파일을 생성합니다.
    """
    
    manifest_data = []
    base_dir = Path(base_path)
    
    # 전체 파일 수 계산
    total_json_files = 0
    all_files = []
    
    print("파일 수 계산 중...")
    for role_dir in base_dir.iterdir():
        if role_dir.is_dir():
            for speaker_dir in role_dir.iterdir():
                if speaker_dir.is_dir():
                    json_files = list(speaker_dir.glob("*.json"))
                    all_files.extend(json_files)
                    total_json_files += len(json_files)
    
    print(f"총 {total_json_files}개의 JSON 파일을 처리합니다.")
    
    # 프로그레스 바를 사용하여 파일 처리
    errors = []
    
    with tqdm(total=total_json_files, desc="Manifest 생성") as pbar:
        for file_path in all_files:
            # JSON 파일 처리
            role_name = file_path.parent.parent.name
            speaker_name = file_path.parent.name
            
            # JSON 파일 읽기 및 처리
            with open(file_path, 'r', encoding='utf-8') as f:
                label_data = json.load(f)
            
            # 음성 파일 경로 생성
            audio_filename = file_path.stem + '.wav'
            audio_path = file_path.parent / audio_filename
            
            # 전사 정보 추출
            transcript = label_data.get('전사정보', {}).get('LabelText', '')
            
            # 파일 정보 추출
            file_info = label_data.get('파일정보', {})
            duration_str = file_info.get('FileLength', '0')
            
            # duration 처리 (문자열일 수 있음)
            duration = float(duration_str) if duration_str else 0.0
            
            # 화자 정보 추출
            speaker_info = label_data.get('화자정보', {})
            gender = speaker_info.get('Gender', '')
            age = speaker_info.get('Age', '')
            region = speaker_info.get('Region', '')
            
            # Manifest 엔트리 생성
            manifest_entry = {
                "audio_filepath": str(audio_path),
                "text": transcript,
                "duration": duration,
                "speaker_id": speaker_name,
                "role": role_name,
                "gender": gender,
                "age": age,
                "region": region
            }
            
            manifest_data.append(manifest_entry)
            pbar.update(1)
    
    # Manifest 파일 저장
    print(f"\n{output_file}에 저장 중...")
    with open(output_file, 'w', encoding='utf-8') as f:
        for entry in manifest_data:
            f.write(json.dumps(entry, ensure_ascii=False) + '\n')
    
    print(f"✅ Manifest 파일이 생성되었습니다: {output_file}")
    print(f"📊 총 {len(manifest_data)}개의 음성 파일이 처리되었습니다.")
    
    # 통계 정보 출력
    roles = {}
    for entry in manifest_data:
        role = entry['role']
        if role not in roles:
            roles[role] = 0
        roles[role] += 1
    
    print("\n📈 역할별 통계:")
    for role, count in roles.items():
        print(f"  {role}: {count:,}개")
    
    return manifest_data


In [ ]:
# 실행 및 결과 확인
base_path = "AI_Hub_비대면진료/비대면_진료를_위한_의료진_및_환자_음성/Validation/medv"

# 개선된 버전으로 실행
print("🚀 개선된 버전으로 Manifest 생성 시작...")
manifest_data = create_manifest_with_progress(base_path, "medical_voice_manifest.jsonl")

# 샘플 데이터 확인
print("\n📝 생성된 데이터 샘플 (처음 3개):")
for i, entry in enumerate(manifest_data[:3]):
    print(f"\n[{i+1}] {entry['role']} - {entry['speaker_id']}")
    print(f"   텍스트: {entry['text']}")
    print(f"   음성파일: {entry['audio_filepath']}")
    print(f"   길이: {entry['duration']}초")
    print(f"   화자정보: {entry['gender']}, {entry['age']}, {entry['region']}")


🚀 개선된 버전으로 Manifest 생성 시작...
파일 수 계산 중...
총 141819개의 JSON 파일을 처리합니다.


Manifest 생성: 100%|██████████| 141819/141819 [08:34<00:00, 275.44it/s]



medical_voice_manifest.jsonl에 저장 중...
✅ Manifest 파일이 생성되었습니다: medical_voice_manifest.jsonl
📊 총 141819개의 음성 파일이 처리되었습니다.

📈 역할별 통계:
  간호사: 42,177개
  의사: 51,898개
  환자: 47,744개

📝 생성된 데이터 샘플 (처음 3개):

[1] 간호사 - HA_0355
   텍스트: 영에서 십 중에서 통증 정도를 골라 보세요.
   음성파일: /home/nas4/DB/AI_Hub_비대면진료/비대면_진료를_위한_의료진_및_환자_음성/Validation/medv/간호사/HA_0355/HA_0355-1833-01-02-F-05-A.wav
   길이: 3.18초
   화자정보: Female, 30~39, 서울/인천/경기

[2] 간호사 - HA_0355
   텍스트: 그 증상으로 얼마 정도 고생하셨어요?
   음성파일: /home/nas4/DB/AI_Hub_비대면진료/비대면_진료를_위한_의료진_및_환자_음성/Validation/medv/간호사/HA_0355/HA_0355-1884-01-02-F-05-A.wav
   길이: 2.76초
   화자정보: Female, 30~39, 서울/인천/경기

[3] 간호사 - HA_0355
   텍스트: 고혈압도 치료 중이신가요?
   음성파일: /home/nas4/DB/AI_Hub_비대면진료/비대면_진료를_위한_의료진_및_환자_음성/Validation/medv/간호사/HA_0355/HA_0355-1711-01-02-F-05-A.wav
   길이: 2.04초
   화자정보: Female, 30~39, 서울/인천/경기


In [ ]:
# 오디오 파일 존재 여부 확인 코드
import json
import os
from pathlib import Path
from tqdm import tqdm

def verify_audio_files(manifest_file, output_file=None):
    """
    Manifest 파일에 포함된 오디오 파일들이 실제로 존재하는지 확인합니다.
    
    Args:
        manifest_file (str): 검증할 manifest 파일 경로
        output_file (str): 존재하는 오디오만 포함한 새 manifest 파일 경로 (선택사항)
    
    Returns:
        dict: 검증 결과 통계
    """
    
    print(f"📂 Manifest 파일 검증 시작: {manifest_file}")
    
    # manifest 파일이 존재하는지 확인
    if not os.path.exists(manifest_file):
        print(f"❌ Manifest 파일을 찾을 수 없습니다: {manifest_file}")
        return None
    
    # manifest 파일 읽기
    manifest_entries = []
    with open(manifest_file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                manifest_entries.append(json.loads(line.strip()))
    
    print(f"📊 총 {len(manifest_entries):,}개의 엔트리를 확인합니다.")
    
    # 검증 결과 저장할 변수들
    existing_files = []
    missing_files = []
    verified_entries = []
    
    # 각 오디오 파일 존재 여부 확인
    with tqdm(total=len(manifest_entries), desc="오디오 파일 검증") as pbar:
        for entry in manifest_entries:
            audio_path = entry.get('audio_filepath', '')
            
            if os.path.exists(audio_path):
                existing_files.append(audio_path)
                verified_entries.append(entry)
            else:
                missing_files.append(audio_path)
            
            pbar.update(1)
    
    # 통계 출력
    print(f"\n✅ 검증 완료!")
    print(f"📈 검증 결과:")
    print(f"  - 존재하는 파일: {len(existing_files):,}개 ({len(existing_files)/len(manifest_entries)*100:.2f}%)")
    print(f"  - 누락된 파일: {len(missing_files):,}개 ({len(missing_files)/len(manifest_entries)*100:.2f}%)")
    
    # 누락된 파일 목록 출력 (처음 10개만)
    if missing_files:
        print(f"\n⚠️  누락된 파일 예시 (처음 10개):")
        for i, missing_file in enumerate(missing_files[:10]):
            print(f"  {i+1}. {missing_file}")
        
        if len(missing_files) > 10:
            print(f"  ... 그 외 {len(missing_files)-10}개 더")
    
    # 검증된 엔트리만으로 새 manifest 파일 생성 (선택사항)
    if output_file and verified_entries:
        print(f"\n💾 검증된 엔트리만으로 새 manifest 파일 생성: {output_file}")
        with open(output_file, 'w', encoding='utf-8') as f:
            for entry in verified_entries:
                f.write(json.dumps(entry, ensure_ascii=False) + '\n')
        print(f"✅ 새 manifest 파일 생성 완료: {len(verified_entries):,}개 엔트리")
    
    # 결과 반환
    result = {
        'total_entries': len(manifest_entries),
        'existing_files': len(existing_files),
        'missing_files': len(missing_files),
        'existing_percentage': len(existing_files)/len(manifest_entries)*100,
        'missing_file_list': missing_files,
        'verified_entries': verified_entries
    }
    
    return result

def analyze_missing_patterns(missing_files):
    """
    누락된 파일들의 패턴을 분석합니다.
    """
    print(f"\n🔍 누락 파일 패턴 분석:")
    
    # 역할별 누락 파일 분석
    role_missing = {}
    speaker_missing = {}
    
    for file_path in missing_files:
        path_obj = Path(file_path)
        
        # 경로에서 역할과 화자 정보 추출
        parts = path_obj.parts
        if len(parts) >= 3:
            # 경로 구조에 따라 조정 필요
            for i, part in enumerate(parts):
                if part in ['nur', 'doc', 'pat']:
                    role_mapping = {'nur': '간호사', 'doc': '의사', 'pat': '환자'}
                    role = role_mapping.get(part, part)
                    
                    if role not in role_missing:
                        role_missing[role] = 0
                    role_missing[role] += 1
                    
                    # 화자 정보 (다음 디렉토리)
                    if i + 1 < len(parts):
                        speaker = parts[i + 1]
                        if speaker not in speaker_missing:
                            speaker_missing[speaker] = 0
                        speaker_missing[speaker] += 1
                    break
    
    # 역할별 누락 통계
    if role_missing:
        print(f"📊 역할별 누락 파일:")
        for role, count in sorted(role_missing.items()):
            print(f"  {role}: {count:,}개")
    
    # 가장 많이 누락된 화자 (상위 10명)
    if speaker_missing:
        print(f"\n🎤 가장 많이 누락된 화자 (상위 10명):")
        sorted_speakers = sorted(speaker_missing.items(), key=lambda x: x[1], reverse=True)
        for i, (speaker, count) in enumerate(sorted_speakers[:10]):
            print(f"  {i+1}. {speaker}: {count:,}개")

def save_missing_files_list(missing_files, output_file="missing_audio_files.txt"):
    """
    누락된 파일 목록을 텍스트 파일로 저장합니다.
    """
    print(f"\n📝 누락된 파일 목록 저장: {output_file}")
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(f"누락된 오디오 파일 목록 (총 {len(missing_files):,}개)\n")
        f.write("=" * 80 + "\n\n")
        
        for i, file_path in enumerate(missing_files, 1):
            f.write(f"{i:6d}. {file_path}\n")
    
    print(f"✅ 누락 파일 목록 저장 완료")


In [ ]:
# 실제 오디오 파일 검증 실행
manifest_file = "medical_voice_manifest.jsonl"

# 1. 기본 검증 (오디오 파일 존재 여부 확인)
print("🚀 오디오 파일 존재 여부 검증 시작...")
verification_result = verify_audio_files(manifest_file)

if verification_result:
    print(f"\n📊 검증 요약:")
    print(f"  전체 엔트리: {verification_result['total_entries']:,}개")
    print(f"  존재하는 파일: {verification_result['existing_files']:,}개")
    print(f"  누락된 파일: {verification_result['missing_files']:,}개")
    print(f"  존재 비율: {verification_result['existing_percentage']:.2f}%")
    
    # 2. 누락 파일 패턴 분석
    if verification_result['missing_files'] > 0:
        analyze_missing_patterns(verification_result['missing_file_list'])
        
        # 3. 누락 파일 목록 저장
        save_missing_files_list(verification_result['missing_file_list'])
    
    # 4. 검증된 엔트리만으로 새 manifest 파일 생성
    if verification_result['existing_files'] > 0:
        print("\n💾 검증된 오디오 파일만으로 새 manifest 파일 생성...")
        verified_manifest_file = "verified_medical_voice_manifest.jsonl"
        
        with open(verified_manifest_file, 'w', encoding='utf-8') as f:
            for entry in verification_result['verified_entries']:
                f.write(json.dumps(entry, ensure_ascii=False) + '\n')
        
        print(f"✅ 검증된 manifest 파일 생성 완료: {verified_manifest_file}")
        print(f"📊 포함된 엔트리: {len(verification_result['verified_entries']):,}개")
else:
    print("❌ 검증 실패")


In [ ]:
# 추가 유틸리티 함수들

def check_audio_file_properties(manifest_file, sample_size=100):
    """
    랜덤하게 선택된 오디오 파일들의 속성을 확인합니다 (길이, 샘플레이트 등).
    """
    import librosa
    import random
    
    print(f"🎵 오디오 파일 속성 확인 (샘플 {sample_size}개)")
    
    # manifest 파일 읽기
    manifest_entries = []
    with open(manifest_file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                manifest_entries.append(json.loads(line.strip()))
    
    # 랜덤 샘플 선택
    sample_entries = random.sample(manifest_entries, min(sample_size, len(manifest_entries)))
    
    audio_properties = []
    
    print("오디오 파일 속성 분석 중...")
    for entry in tqdm(sample_entries):
        audio_path = entry['audio_filepath']
        
        if os.path.exists(audio_path):
            # librosa로 오디오 정보 추출
            duration = librosa.get_duration(filename=audio_path)
            y, sr = librosa.load(audio_path, sr=None)
            
            properties = {
                'file_path': audio_path,
                'duration_librosa': duration,
                'duration_manifest': entry.get('duration', 0),
                'sample_rate': sr,
                'length_samples': len(y),
                'role': entry.get('role', ''),
                'speaker_id': entry.get('speaker_id', '')
            }
            
            audio_properties.append(properties)
    
    if audio_properties:
        # 통계 출력
        durations = [p['duration_librosa'] for p in audio_properties]
        sample_rates = [p['sample_rate'] for p in audio_properties]
        
        print(f"\n📊 오디오 속성 통계 (샘플 {len(audio_properties)}개):")
        print(f"  평균 길이: {sum(durations)/len(durations):.2f}초")
        print(f"  최소 길이: {min(durations):.2f}초")
        print(f"  최대 길이: {max(durations):.2f}초")
        print(f"  샘플레이트: {set(sample_rates)}")
        
        # manifest의 duration과 실제 duration 비교
        duration_diffs = [abs(p['duration_librosa'] - p['duration_manifest']) for p in audio_properties]
        avg_diff = sum(duration_diffs) / len(duration_diffs)
        print(f"  manifest vs 실제 길이 차이: 평균 {avg_diff:.3f}초")
        
        return audio_properties
    else:
        print("❌ 분석할 수 있는 오디오 파일이 없습니다.")
        return []

def generate_summary_report(verification_result, output_file="audio_verification_report.txt"):
    """
    검증 결과에 대한 상세 보고서를 생성합니다.
    """
    print(f"📝 검증 보고서 생성: {output_file}")
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("=" * 80 + "\n")
        f.write("비대면 진료 음성 데이터 Manifest 검증 보고서\n")
        f.write("=" * 80 + "\n\n")
        
        f.write(f"검증 일시: {os.popen('date').read().strip()}\n\n")
        
        f.write("1. 검증 요약\n")
        f.write("-" * 40 + "\n")
        f.write(f"전체 엔트리 수: {verification_result['total_entries']:,}개\n")
        f.write(f"존재하는 파일: {verification_result['existing_files']:,}개\n")
        f.write(f"누락된 파일: {verification_result['missing_files']:,}개\n")
        f.write(f"존재 비율: {verification_result['existing_percentage']:.2f}%\n\n")
        
        if verification_result['missing_files'] > 0:
            f.write("2. 권장사항\n")
            f.write("-" * 40 + "\n")
            f.write("• 누락된 오디오 파일들을 확인하고 복구하세요.\n")
            f.write("• verified_medical_voice_manifest.jsonl 파일을 사용하여 학습을 진행하세요.\n")
            f.write("• missing_audio_files.txt에서 누락된 파일 목록을 확인하세요.\n\n")
        else:
            f.write("2. 검증 결과\n")
            f.write("-" * 40 + "\n")
            f.write("✅ 모든 오디오 파일이 정상적으로 존재합니다.\n\n")
    
    print(f"✅ 검증 보고서 생성 완료")

def quick_manifest_stats(manifest_file):
    """
    Manifest 파일의 기본 통계를 빠르게 확인합니다.
    """
    print(f"📊 {manifest_file} 기본 통계")
    
    manifest_entries = []
    with open(manifest_file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                manifest_entries.append(json.loads(line.strip()))
    
    # 기본 통계
    total_entries = len(manifest_entries)
    total_duration = sum(entry.get('duration', 0) for entry in manifest_entries)
    
    # 역할별 분포
    role_counts = {}
    for entry in manifest_entries:
        role = entry.get('role', 'Unknown')
        role_counts[role] = role_counts.get(role, 0) + 1
    
    print(f"  총 엔트리 수: {total_entries:,}개")
    print(f"  총 오디오 길이: {total_duration:.2f}초 ({total_duration/3600:.2f}시간)")
    print(f"  평균 오디오 길이: {total_duration/total_entries:.2f}초")
    print(f"  역할별 분포:")
    for role, count in sorted(role_counts.items()):
        print(f"    {role}: {count:,}개 ({count/total_entries*100:.1f}%)")
    
    return {
        'total_entries': total_entries,
        'total_duration': total_duration,
        'role_distribution': role_counts
    }


In [ ]:
# 사용 예시
print("🎯 오디오 파일 검증 도구 사용 예시")
print("=" * 60)

# 기본 사용법
print("\n1️⃣ 기본 검증:")
print("verification_result = verify_audio_files('medical_voice_manifest.jsonl')")

print("\n2️⃣ 검증 + 새 manifest 파일 생성:")
print("verification_result = verify_audio_files('medical_voice_manifest.jsonl', 'verified_manifest.jsonl')")

print("\n3️⃣ 누락 파일 패턴 분석:")
print("analyze_missing_patterns(verification_result['missing_file_list'])")

print("\n4️⃣ 오디오 속성 확인 (librosa 필요):")
print("audio_props = check_audio_file_properties('verified_manifest.jsonl', sample_size=50)")

print("\n5️⃣ 빠른 통계 확인:")
print("stats = quick_manifest_stats('medical_voice_manifest.jsonl')")

print("\n6️⃣ 검증 보고서 생성:")
print("generate_summary_report(verification_result)")

print("\n" + "=" * 60)
print("📝 참고사항:")
print("• 검증 후 'verified_medical_voice_manifest.jsonl' 파일을 학습에 사용하세요")
print("• 누락된 파일 목록은 'missing_audio_files.txt'에서 확인 가능합니다")
print("• 큰 데이터셋의 경우 검증에 시간이 오래 걸릴 수 있습니다")
print("• librosa가 설치되어 있어야 오디오 속성 확인 기능을 사용할 수 있습니다")


In [3]:
# 추가 분석 및 필터링 기능
import pandas as pd

def analyze_manifest_data(manifest_data):
    """생성된 manifest 데이터를 분석합니다."""
    
    df = pd.DataFrame(manifest_data)
    
    print("📊 상세 통계 분석")
    print("=" * 50)
    
    # 역할별 통계
    print("\n🎭 역할별 분포:")
    role_counts = df['role'].value_counts()
    for role, count in role_counts.items():
        print(f"  {role}: {count:,}개 ({count/len(df)*100:.1f}%)")
    
    # 성별 분포
    print("\n👥 성별 분포:")
    gender_counts = df['gender'].value_counts()
    for gender, count in gender_counts.items():
        print(f"  {gender}: {count:,}개 ({count/len(df)*100:.1f}%)")
    
    # 연령대 분포
    print("\n🎂 연령대 분포:")
    age_counts = df['age'].value_counts()
    for age, count in age_counts.items():
        print(f"  {age}: {count:,}개 ({count/len(df)*100:.1f}%)")
    
    # 지역별 분포
    print("\n🗺️ 지역별 분포:")
    region_counts = df['region'].value_counts()
    for region, count in region_counts.items():
        print(f"  {region}: {count:,}개 ({count/len(df)*100:.1f}%)")
    
    # 음성 길이 통계
    print("\n⏱️ 음성 길이 통계:")
    print(f"  평균: {df['duration'].mean():.2f}초")
    print(f"  최소: {df['duration'].min():.2f}초")
    print(f"  최대: {df['duration'].max():.2f}초")
    print(f"  전체: {df['duration'].sum():.2f}초 ({df['duration'].sum()/3600:.2f}시간)")
    
    # 텍스트 길이 통계
    df['text_length'] = df['text'].str.len()
    print("\n📝 텍스트 길이 통계:")
    print(f"  평균: {df['text_length'].mean():.1f}자")
    print(f"  최소: {df['text_length'].min()}자")
    print(f"  최대: {df['text_length'].max()}자")
    
    return df

def create_filtered_manifest(df, filters, output_file):
    """조건에 맞는 데이터만 필터링하여 새로운 manifest 파일 생성"""
    
    filtered_df = df.copy()
    
    # 필터 적용
    for key, values in filters.items():
        if key in df.columns:
            if isinstance(values, list):
                filtered_df = filtered_df[filtered_df[key].isin(values)]
            else:
                filtered_df = filtered_df[filtered_df[key] == values]
    
    print(f"필터링 결과: {len(filtered_df):,}개 데이터")
    
    # 필터링된 데이터를 manifest 파일로 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        for _, row in filtered_df.iterrows():
            entry = row.to_dict()
            # text_length 컬럼 제거 (manifest에 불필요)
            if 'text_length' in entry:
                del entry['text_length']
            f.write(json.dumps(entry, ensure_ascii=False) + '\n')
    
    print(f"필터링된 manifest 파일 저장: {output_file}")
    
    return filtered_df


In [ ]:
# 데이터 분석 및 필터링 예시
if 'manifest_data' in locals():
    # 상세 분석 실행
    df = analyze_manifest_data(manifest_data)
    
    print("\n" + "="*50)
    print("🔍 필터링 예시")
    print("="*50)
    
    # 예시 1: 여성 의사 데이터만 추출
    filters_female_doctor = {
        'role': '의사',
        'gender': 'Female'
    }
    filtered_df1 = create_filtered_manifest(df, filters_female_doctor, "female_doctor_manifest.jsonl")
    
    # 예시 2: 20-30대 환자 데이터만 추출
    filters_young_patient = {
        'role': '환자',
        'age': ['20~29', '30~39']
    }
    filtered_df2 = create_filtered_manifest(df, filters_young_patient, "young_patient_manifest.jsonl")
    
    # 예시 3: 특정 지역 데이터만 추출
    filters_region = {
        'region': '서울/경기/인천'
    }
    if len(df[df['region'] == '서울/경기/인천']) > 0:
        filtered_df3 = create_filtered_manifest(df, filters_region, "seoul_region_manifest.jsonl")
    else:
        print("서울/경기/인천 지역 데이터가 없습니다.")
else:
    print("먼저 manifest_data를 생성해주세요.")
